#setup

In [2]:
!pip install pyodbc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.5/343.5 kB 5.2 MB/s eta 0:00:00


In [3]:
#Parameters


#configuring odbc
import pyodbc

server = 'wildfire-data-analytics-server.database.windows.net'
database = 'wildfire-data-analytics'
username = 'lucasvittal'
password = 'Lv300699@$'
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

Error: ignored

# Análise das queimadas na última década




Na última década o fenômenos de incêndios florestais,mas comumente mencionadas como queimadas aqui no brasil, tem chamado cada vez mais atenção mundialmente. As queimadas estão cada vez mais frequentes e intensas e tem causada devastações e perdas que tem  quebrado recordes sobre tudo nos últimos anos. Tendo em vista as catástrofes ocorridas noa Havaí e  no Canadá esse ano tal fenômeno chamou minha atenção de modo a senti-me engajado em tentar entender por meio dos dados o porque tal fenômeno anda  tão frequente e intenso e as verdadeira magnitudes de seu impacto na vida das pessoas que vivem nas área afetadas. Portanto, a ideia nesse trabalho é obter os dados necessários para responder as seguintes indagações:


- Sobre evolução das queimadas na última década:

  - Como tem sido a evolução das queimadas nessa última decada ?
  - Como tem sido a evolução em intensidade dos incêndia nessa última década?

- Análise do perfil das localidades onde as queimadas ocorrem com maior frequência e intensidade:

  - Onde tem acontecido os incendios com mais intensidade ?
  - Qual a caracterítica desses locais em termos de localização espacial, temperatura, umidade, vegetação?


- Sobre Perdas em termos de vegestação e  impactado na vida das pessoas:

  -  Perdas médias globais de vegetação
  -  Perdas média de vegetação por continente
  -  Perda média  de vegetação nos países afetados
  - Perdas ecômicas ( média global, continental, e por país) devido as queimadas
  - impacto na qualidade do ar

-  Sobre a relação entre as mudanças climáticas e o aumento das queimadas:
  
  - Emissões de carbono
  - aumento da temperatura
  - Relação entre aumento da temperatura global e local por país
  


  
  Sendo assim o objetivo deste trabalho é responder as pesguntas acima de  modo que o compilado de resposta forme uma resposta consistente sobre a pergunta principal:

  ***Porque as queimadas tem aumentado tanto em frequência e intensidade na última década ?***

  De modo a compreender todos os fatores a ela elencados. Para se chegar a esse compilado de respostas a ideia é usar os conhecimento  aprendidos essa Sprint realizando a coleta dos dados, a sua modelagem de forma adequada adequada e construção de um catálogo de dados, a carga desses dados numa infraestrutura de nuvem e construção de um pipeline de tratamento,  a análise dos dados para finalmente se obter a resposta e com isso se tirar um conclusão final a cerca da principal resposta buscada.
  
  


# Busca e coleta de dados

Buscando responde cada uma das perguntas acima, buscou-se dados  referentes a cada um do aspectos em que se entendeu como parte da pergunta principal. Sendo assim dos dados que foram encontrados, os seguintes abaixo foram entendidos como relevates para as respostas buscadas:


**evolução das queimadas na última década**  
   - [Monthly burned area [ha] by landcover class for years 2002-2019 - EU](https://gwis.jrc.ec.europa.eu/apps/country.profile/downloads)
   

**Característica de vegetação por localização**
  - [LANDCOVER DATASET - OCDE](https://stats.oecd.org/Index.aspx?DataSetCode=LAND_COVER)


**Média temperatura por país**
  - [Temperatura all Countries](https://www.kaggle.com/datasets/subhamjain/temperature-of-all-countries-19952020/code)
  -  [Mean global temperature  - Berkeley University](https://berkeleyearth.org/data/)
  - [Mean Temperature Cites on Canada - climate data](https://climatedata.ca/download/#station-download)
  - [Average monthly temperature by us state - kaggle](https://www.kaggle.com/datasets/justinrwong/average-monthly-temperature-by-us-state)
  - [Hawaii monthly average temperatures kahului - nceii noa](https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/city/time-series/USW00022516)


**Média anual de precipitação**
 - [precipation per country - World Bank](https://data.worldbank.org/indicator/AG.LND.PRCP.MM?end=2020&start=1961)


**Perdas em aspecto econômicos e mortes devido as queimadas**

  - [Global economic Losses and deaths due to wildfires -  World Bank](https://data.worldbank.org/indicator/AG.LND.PRCP.MM?end=2020&start=1961)
  - [Disaster Database -  EM Data](https://public.emdat.be/data)


   **Emissiões Anuais de carbono por país**
  -  [ CO2 Emissions - Ourworld Daata](https://ourworldindata.org/co2-dataset-sources)
  -  [Greenhouse gas emissions - OCDE](https://stats.oecd.org/Index.aspx?DataSetCode=air_ghg)
  -  [ CO2 Emissions - World Bank](https://data.worldbank.org/indicator/EN.ATM.CO2E.PC)

Tendo em vista que o que se deseja obter como fonte para camada de visualização é um Datawarehouse, fez-se então uma análise mais aprofundadada dos dados coletados selecionando os dados que de fato eram relevantes para a análise a ser realizada. Muitos dos dados encontrados eram redudantes, porém algumas tabelas apresentavam-se mais completas do que outras e foram justamente essas tabelas as quais foi dada a prioriadade na seleção. Uma vez selecionadas as  tabelas, quebrou-se essas tabelas em uma tabela fato e as tabelas  dimensões correspondentes. Tal divisáo fora feita considerando as principais métricas que deveriam ser obtidas as quais tinham que estar na tabela fato, já as tabelas dimensões foram definidas com campos que definiam algum aspecto de granularidade relativa as métricas principais que deveriam ser obitidas a partir da tabela fato. Muitas tabelas  fatos acabaram por compartilha as mesma dimensões então a mesma pode ser aproveitada para outra análise de métrica ao qul se desejou  realizar. A partir desse processo obteve-se um modelo de DW abaixo, onde ao todo se obteve um total de 6 tabelas fato:




**bold text**# Modelagem Dos Dados

Tendo em vista que o que se deseja obter como fonte para camada de visualização é um Datawarehouse, fez-se então uma análise mais aprofundadada dos dados coletados selecionando os dados que de fato eram relevantes para a análise a ser realizada. Muitos dos dados encontrados eram redudantes, porém algumas tabelas apresentavam-se mais completas do que outras e foram justamente essas tabelas as quais foi dada a prioriadade na seleção. Uma vez selecionadas as  tabelas, quebrou-se essas tabelas em uma tabela fato e as tabelas  dimensões correspondentes. Tal divisáo fora feita considerando as principais métricas que deveriam ser obtidas as quais tinham que estar na tabela fato, já as tabelas dimensões foram definidas com campos que definiam algum aspecto de granularidade relativa as métricas principais que deveriam ser obitidas a partir da tabela fato. Muitas tabelas  fatos acabaram por compartilha as mesma dimensões então a mesma pode ser aproveitada para outra análise de métrica ao qul se desejou  realizar. A partir desse processo obteve-se um modelo de DW abaixo, onde ao todo se obteve um total de 6 tabelas fato. O esquema relacional construido a partir desta modelagem pode ser acessado no link abaixo:

[WildFire_Analysis_DW_Schema](https://drive.google.com/file/d/1L1eHUI49MDDmFsBoyL3AfaJzykHaSKl0/view?usp=sharing)




## Catálogo de dados

In [ ]:
import os



In [ ]:
import json
def read_json(base_path, filename):
  with open(base_path +filename) as q:
      json_file =  json.load(q)
  return json_file

In [ ]:
import pandas as pd
#read json and save as excel file

folder_content = os.listdir(JSON_CATALOGS_PATH)
json_filenames = [json_file for json_file in folder_content if os.path.isfile(os.path.join(JSON_CATALOGS_PATH, json_file))]
json_files = [read_json(JSON_CATALOGS_PATH, filename) for filename in json_filenames]
for json_file,filename in zip(json_files, json_filenames):
  pd.DataFrame(json_file).to_excel(TABLE_CATALOGS_PATH + 'excel/' + filename.replace('.json', '.xlsx'),)


In [ ]:
json_filenames

['ocde_carbon_emission_per_country_catalog.json',
 'ncei_hawaii_kahului_avg_temperature.json',
 'kaggle_city_temperature_catalog.json',
 'cd_climate_daily_canada_per_state_catalog.json',
 'ocde_land_cover_catalog.json',
 'kaggle_US_average_monthly_temperature_by_state_1950_2022_catalog.json',
 'emdat_disasters_impact_uidds2nac_catalog.json',
 'ncei_mean_global_temperatues_1850_2023_land_sea_catalog.json',
 'gwis_burned_area_full_dataset_2002_2019_catalog.json',
 'mb_avg_preciptation_per_year_catalog.json']

In [ ]:
#read json and save as csv's files

folder_content = os.listdir(JSON_CATALOGS_PATH)
json_filenames = [json_file for json_file in folder_content if os.path.isfile(os.path.join(JSON_CATALOGS_PATH, json_file))]
json_files = [read_json(JSON_CATALOGS_PATH, filename) for filename in json_filenames]
for json_file,filename in zip(json_files, json_filenames):
  pd.DataFrame(json_file).to_csv(TABLE_CATALOGS_PATH + 'csv/' + filename.replace('.json', '.csv'), encoding='utf-8')